In [1]:
import numpy as np
import tensorflow as tf
import re
import time

In [2]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [3]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [4]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

In [5]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [8]:
clean_questions = []
clean_answers = []
for question in questions:
    clean_questions.append(clean_text(question))
   

    

In [9]:
for answer in answers:
    clean_answers.append(clean_text(answer))
    

In [10]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [11]:
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1


In [12]:
word2count

{'can': 15975,
 'we': 40502,
 'make': 6747,
 'this': 33579,
 'quick': 337,
 'roxanne': 1,
 'korrine': 1,
 'and': 65607,
 'andrew': 56,
 'barrett': 19,
 'are': 54580,
 'having': 1217,
 'an': 9482,
 'incredibly': 60,
 'horrendous': 4,
 'public': 368,
 'break': 895,
 'up': 16050,
 'on': 27238,
 'the': 140644,
 'quad': 2,
 'again': 3193,
 'well': 14111,
 'i': 204535,
 'thought': 4550,
 'would': 20009,
 'start': 1656,
 'with': 24961,
 'pronunciation': 2,
 'if': 18952,
 'that': 67092,
 'is': 79611,
 'okay': 6097,
 'you': 212942,
 'not': 32325,
 'hacking': 18,
 'gagging': 9,
 'spitting': 16,
 'part': 1417,
 'please': 3209,
 'asking': 746,
 'me': 44905,
 'out': 18468,
 'so': 19059,
 'cute': 272,
 'what': 55199,
 'your': 29938,
 'name': 3122,
 'no': 27575,
 "it's": 25845,
 'my': 29687,
 'fault': 482,
 "didn't": 8735,
 'have': 46595,
 'a': 102010,
 'proper': 138,
 'introduction': 19,
 'cameron': 35,
 'thing': 5731,
 'am': 37862,
 'at': 15290,
 'mercy': 68,
 'of': 56296,
 'particularly': 111,
 'h

In [13]:
threshold_questions = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1


In [14]:
threshold_answers = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1


In [15]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [16]:
answersints2word = {w_i:w for w,w_i in answerswords2int.items()}

In [17]:
for i in range(len(clean_answers)):
    clean_answers[i] += '<EOS>'

In [18]:
clean_answers

['well i thought we would start with pronunciation if that is okay with you<EOS>',
 'not the hacking and gagging and spitting part  please<EOS>',
 "okay then how 'bout we try out some french cuisine  saturday  night<EOS>",
 'forget it<EOS>',
 'cameron<EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does<EOS>',
 'seems like she could get a date easy enough<EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something<EOS>',
 'that is a shame<EOS>',
 'let me see what i can do<EOS>',
 'right  see  you are ready for the quiz<EOS>',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone<EOS>",
 "that is because it's such a nice one<EOS>",
 'forget french<EOS>',
 "well there's some

In [21]:
questions_to_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_to_int.append(ints) 

In [23]:
answers_to_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_to_int.append(ints) 


In [24]:
answers_to_int

[[17, 18, 19, 1, 20, 21, 22, 8824, 23, 24, 25, 26, 22, 8824],
 [28, 15, 8824, 5, 8824, 5, 8824, 29, 8824],
 [26, 73, 97, 1533, 1, 860, 33, 482, 387, 8824, 210, 8824],
 [245, 8824],
 [8824],
 [15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8824,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  8824],
 [399, 77, 60, 84, 129, 45, 58, 865, 8824],
 [8824,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  8824],
 [24, 25, 45, 8824],
 [287, 32, 91, 36, 18, 0, 8824],
 [90, 91, 27, 7, 92, 93, 15, 8824],
 [18,
  94,
  95,
  65,
  96,
  97,
  65,
  98,
  24,
  99,
  18,
  95,
  65,
  96,
  100,
  101,
  77,
  102,
  15,
  103,
  104,
  7,
  97,
  105,
  61,
  106,
  107,
  108,
  77,
  109,
  18,
  44,
  110,
  111,
  41,
  112,
  113,
  65,
  114,
  33,
  41,
  89,
  65,
  8824],
 [24, 25, 116, 40, 117, 45, 118, 8824],
 [245, 8824],
 [17, 316, 115, 18, 194, 239, 66, 882

In [25]:
questions_to_int

[[0,
  1,
  2,
  3,
  4,
  8824,
  8824,
  5,
  6,
  8824,
  7,
  8,
  9,
  10,
  8824,
  11,
  12,
  13,
  14,
  15,
  8824,
  16],
 [17, 18, 19, 1, 20, 21, 22, 8824, 23, 24, 25, 26, 22, 27],
 [28, 15, 8824, 5, 8824, 5, 8824, 29, 30],
 [27, 7, 31, 32, 33, 24, 25, 34, 35, 36, 25, 37, 38, 16],
 [39, 39, 40, 41, 42, 1, 43, 44, 45, 46, 8824],
 [47],
 [15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8824,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61],
 [62],
 [8824,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81],
 [82, 23, 83, 1, 84, 85, 86, 45, 87],
 [8824, 88, 8824, 3, 25, 41, 89],
 [90, 91, 27, 7, 92, 93, 15, 8824],
 [18,
  94,
  95,
  65,
  96,
  97,
  65,
  98,
  24,
  99,
  18,
  95,
  65,
  96,
  100,
  101,
  77,
  102,
  15,
  103,
  104,
  7,
  97,
  105,
  61,
  106,
  107,
  108,
  77,
  109,
  18,
  44,
  110,
  111,
  41,
  112,
  113,
 

In [28]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25 + 1):
    for i in enumerate(questions_to_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_to_int[i[0]])